In [ ]:
import pyodbc
import pandas as pd
import warnings
import sqlite3
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [ ]:
def get_data_type(x):
    type_ranges = {
        'int8': (-128, 127),
        'int16': (-32768, 32767),
        'int32': (-2147483648, 2147483647),
        'int64': (-9223372036854775808, 9223372036854775807),
        'float32': (-3.4028235e38, 3.4028235e38),
        'float64': (-1.7976931348623157e308, 1.7976931348623157e308)
    }

    for data_type, value_range in type_ranges.items():
        if value_range[0] <= x <= value_range[1]:
            return data_type

    return 'not a valid data type'

In [ ]:
import pyodbc
import sqlite3
import pandas as pd

# Data cleaning
col_exc = ['DYCOL03', 'TakmilDescC']
pattern = r'^\D|^$'
passed_zero = {}
empty_table = []
def main(file, file_name):
    # Constants
    db_dir = file
    msa_drivers = [x for x in pyodbc.drivers() if 'ACCESS' in x.upper()]
    con_string = f'DRIVER={msa_drivers[0]};DBQ={db_dir}'  # msa_drivers[0] = 'Microsoft Access Driver (*.mdb, *.accdb)'
    print('db_dir and con_string created.')

    with pyodbc.connect(con_string) as conn_pyodbc, sqlite3.connect(f'{file_name}.db') as conn_sqlite3:
        cursor = conn_pyodbc.cursor()
        print('conn_pyodbc, cursor and conn_sqlite3 created')

        # Extract table names and initial columns
        table_names = [x.table_name for x in cursor.tables() if x.table_type == 'TABLE']
        tables_cols = {table: [x.column_name for x in cursor.columns(table=table)] for table in table_names}
        print('table names and initial columns extracted.')

        # Filter columns and exclude tables
        exclude_tb = list(filter(lambda table: table.endswith('P3S13') or table.endswith('Data'), table_names))
        final_tables_cols = {key: [value for value in tables_cols[key] if value not in col_exc]
                              if key in exclude_tb else tables_cols[key] for key in tables_cols}
        print('columns filtered.')

        # Process columns
        for table, cols in final_tables_cols.items():
            data_types = {}
            df = pd.read_sql_query(f"SELECT * FROM {table}", conn_pyodbc)
            if len(df) == 0:
                empty_table.append(table)
                continue
            print(f'df for {table}, created')
            for col in cols:
                print(f'process on {col}')
                if df[col].dtype == 'object':
                    df[col] = df[col].str.lstrip('0').replace(pattern, 0, regex=True)
                else:
                    passed_zero[table] = col
                    pass
                maxx = df[col].astype('int64').unique().max()
                data_type = get_data_type(maxx)
                data_types[col] = data_type
                df[col] = df[col].astype(data_type)
                print(f'{col} column data type casted.')
            df.to_sql(table, conn_sqlite3, if_exists='replace', index=False, dtype=data_types)
            data_types = {}
            print(f'{table} table inserted to sqlite3 db.')
            clear_output(wait=True)

    print('conn_pyodbc and conn_sqlite3 closed.')


In [ ]:
import glob

# specify the directory to search in
directory = r"your_directory"

# find all .mdb files in the directory
mdb_files = glob.glob(directory + '/*.mdb')
for file in mdb_files:
    file_name = str(file).split('\\')[-1][:-4]
    main(file, file_name)
